In [11]:
import pandas as pd
import numpy as np
import time

weather = pd.read_csv('data/weather.csv')
pallet_history = pd.read_csv('data/Pallet_history_Gold_Spike.csv')
inbound = pd.read_csv('data/inbound_loads.csv')
outbound = pd.read_csv('data/outbound_laods.csv')
demand = pd.read_csv('data/demand_kWtrain_val.csv')

In [12]:
#################
# Preprocess inbound csv:
#################
print('\n #####\nInbound: \n ##### \n')
inbound_post = inbound[inbound.carrier_code != 'CANCEL']
inbound_post = inbound[inbound.carrier_code != '']

inbound_post['truck_signin_datetime'] = pd.to_datetime(inbound_post['truck_signin_datetime'])

# Compute delta times
inbound_load_time = pd.to_datetime(inbound_post['load_finish_datetime']) - pd.to_datetime(inbound_post['load_start_datetime'])
inbound_truck_time = pd.to_datetime(inbound_post['truck_signin_datetime']) - pd.to_datetime(inbound_post['signout_datetime'])

# Drop unnecessary columns
inbound_post = inbound_post.drop(['Unnamed: 0', 'warehouse_order_number', 'customer_code', 'load_reference_number', 'carrier_code', 'weight_uom', 'load_finish_datetime', 'load_start_datetime', 'dock_door_number', 'trailer_number', 'signout_datetime'], axis=1)

# Add time deltas
inbound_post['load_time'] = inbound_load_time
inbound_post['truck_time'] = inbound_truck_time

print(inbound_post.columns)

inbound_post['load_time'] = inbound_post['load_time'].dt.seconds
inbound_post['truck_time'] = inbound_post['truck_time'].dt.seconds

print('With NaN:', inbound_post.shape)

# Drop rows with >0 NaN values
inbound_post_nan = inbound_post.dropna().reset_index(drop=True)

print('Without NaN:', inbound_post_nan.shape)

#################
# Preprocess outbound csv:
#################
print('\n #####\nOutbound: \n ##### \n')
outbound_post = outbound[outbound.carrier_code != 'CANCEL']
outbound_post = outbound[outbound.carrier_code != 'VOID']
outbound_post = outbound[outbound.carrier_code != '']

outbound_post['truck_signin_datetime'] = pd.to_datetime(outbound_post['truck_signin_datetime'])

# Compute delta times
outbound_load_time = pd.to_datetime(outbound_post['load_finish_datetime']) - pd.to_datetime(outbound_post['load_start_datetime'])
outbound_truck_time = pd.to_datetime(outbound_post['truck_signin_datetime']) - pd.to_datetime(outbound_post['signout_datetime'])

# Drop unnecessary columns
outbound_post = outbound_post.drop(['Unnamed: 0', 'warehouse_order_number', 'customer_code', 'load_reference_number', 'carrier_code', 'weight_uom', 'load_finish_datetime', 'load_start_datetime', 'dock_door_number', 'trailer_number', 'signout_datetime'], axis=1)

# Add time deltas
outbound_post['load_time'] = outbound_load_time
outbound_post['truck_time'] = outbound_truck_time

print(outbound_post.columns)

outbound_post['load_time'] = outbound_post['load_time'].dt.seconds
outbound_post['truck_time'] = outbound_post['truck_time'].dt.seconds

print('With NaN:', outbound_post.shape)

# Drop rows with >0 NaN values
outbound_post_nan = outbound_post.dropna().reset_index(drop=True)

print('Without NaN:', outbound_post_nan.shape)

#################
# Preprocess demand csv:
#################
print('\n #####\nDemand:  \n ##### \n')

demand['datetime_local'] = pd.to_datetime(demand['datetime_local'])

end_known_idx = demand[demand.demand_kW > 1].index[-1]
train_val_split = 0.7 # 70% train, 30% val
end_train_idx = int((train_val_split) * end_known_idx)
demand_train = demand[0:end_train_idx-1]
demand_val = demand[end_train_idx:end_known_idx]

print('Full dataset:', demand.shape)
print('Answers known until index: ', end_known_idx)
print(f'Training set, {int(train_val_split*100)}%:', demand_train.shape)
print(f'Validation set, {int(100-train_val_split*100)}%:', demand_val.shape)

#################
# Preprocess weather csv:
#################
print('\n #####\nWeather:  \n ##### \n')

weather_post = weather.copy()
UTC6 = pd.to_datetime(weather_post['datetime_UTC']) - pd.Timedelta(hours=6)
weather_post['datetime_america'] = UTC6
weather_post = weather_post.drop('datetime_UTC', axis=1)
weather_post = weather_post.drop('datetime', axis=1)

print(weather_post.head())
print(weather_post.columns)
print(weather_post.shape)



 #####
Inbound: 
 ##### 

Index(['front_temperature', 'middle_temperature', 'back_temperature',
       'net_weight', 'case_quantity', 'pallet_count', 'truck_signin_datetime',
       'load_time', 'truck_time'],
      dtype='object')
With NaN: (56146, 9)
Without NaN: (8761, 9)

 #####
Outbound: 
 ##### 

Index(['net_weight', 'case_quantity', 'pallet_count', 'truck_signin_datetime',
       'load_time', 'truck_time'],
      dtype='object')
With NaN: (112363, 6)
Without NaN: (96704, 6)

 #####
Demand:  
 ##### 

Full dataset: (365349, 3)
Answers known until index:  273987
Training set, 70%: (191789, 3)
Validation set, 30%: (82197, 3)

 #####
Weather:  
 ##### 

   Unnamed: 0  Relative Humidity  Temperature  hour    datetime_america
0           0              50.37         53.6    18 2018-12-31 18:00:00
1           1              50.37         53.6    18 2018-12-31 18:05:00
2           2              50.37         53.6    18 2018-12-31 18:10:00
3           3              50.37         53.6 

# Training set

Merge inbound with demand

In [13]:
inbound_post_nan.sort_values("truck_signin_datetime", inplace=True)
demand_inbound_merge = pd.merge_asof(demand_train, inbound_post_nan, left_on='datetime_local', right_on='truck_signin_datetime', direction='nearest')

demand_inbound_merge_numerical = demand_inbound_merge.copy()
demand_inbound_merge_numerical['datetime_local'] = demand_inbound_merge_numerical['datetime_local'].apply(lambda x: time.mktime(x.timetuple()))
demand_inbound_merge_numerical['truck_signin_datetime'] = demand_inbound_merge_numerical['truck_signin_datetime'].apply(lambda x: time.mktime(x.timetuple()))

merge with outbound

In [14]:
outbound_post_nan.sort_values("truck_signin_datetime", inplace=True)
outbound_merge = pd.merge_asof(demand_inbound_merge, outbound_post_nan, left_on='datetime_local', right_on='truck_signin_datetime', direction='nearest', suffixes=('_inbound', '_outbound'))

outbound_merge_numerical = outbound_merge.copy()
outbound_merge_numerical['datetime_local'] = outbound_merge_numerical['datetime_local'].apply(lambda x: time.mktime(x.timetuple()))
outbound_merge_numerical['truck_signin_datetime_inbound'] = outbound_merge_numerical['truck_signin_datetime_inbound'].apply(lambda x: time.mktime(x.timetuple()))
outbound_merge_numerical['truck_signin_datetime_outbound'] = outbound_merge_numerical['truck_signin_datetime_outbound'].apply(lambda x: time.mktime(x.timetuple()))

merge with weather

In [15]:
weather_post.sort_values("datetime_america", inplace=True)
weather_merge = pd.merge_asof(outbound_merge, weather_post, left_on='datetime_local', right_on='datetime_america', direction='nearest')

weather_merge_numerical = weather_merge.copy()
weather_merge_numerical['datetime_local'] = weather_merge_numerical['datetime_local'].apply(lambda x: time.mktime(x.timetuple()))
weather_merge_numerical['truck_signin_datetime_inbound'] = weather_merge_numerical['truck_signin_datetime_inbound'].apply(lambda x: time.mktime(x.timetuple()))
weather_merge_numerical['truck_signin_datetime_outbound'] = weather_merge_numerical['truck_signin_datetime_outbound'].apply(lambda x: time.mktime(x.timetuple()))
weather_merge_numerical['datetime_america'] = weather_merge_numerical['datetime_america'].apply(lambda x: time.mktime(x.timetuple()))


Removing the date columns, because their magnitude may be interfering with the regression.

In [16]:
no_dates_merged = weather_merge_numerical.drop(['datetime_local', 'truck_signin_datetime_inbound', 'truck_signin_datetime_outbound', 'datetime_america'], axis=1)
nonan_train = no_dates_merged.dropna().reset_index(drop=True)

nonan_train.to_csv('preprocessed_data/preprocessed_train_data.csv', index=False)

# Validation set

In [17]:
inbound_post_nan.sort_values("truck_signin_datetime", inplace=True)
demand_inbound_merge = pd.merge_asof(demand_val, inbound_post_nan, left_on='datetime_local', right_on='truck_signin_datetime', direction='nearest')

demand_inbound_merge_numerical = demand_inbound_merge.copy()
demand_inbound_merge_numerical['datetime_local'] = demand_inbound_merge_numerical['datetime_local'].apply(lambda x: time.mktime(x.timetuple()))
demand_inbound_merge_numerical['truck_signin_datetime'] = demand_inbound_merge_numerical['truck_signin_datetime'].apply(lambda x: time.mktime(x.timetuple()))

outbound_post_nan.sort_values("truck_signin_datetime", inplace=True)
outbound_merge = pd.merge_asof(demand_inbound_merge, outbound_post_nan, left_on='datetime_local', right_on='truck_signin_datetime', direction='nearest', suffixes=('_inbound', '_outbound'))

outbound_merge_numerical = outbound_merge.copy()
outbound_merge_numerical['datetime_local'] = outbound_merge_numerical['datetime_local'].apply(lambda x: time.mktime(x.timetuple()))
outbound_merge_numerical['truck_signin_datetime_inbound'] = outbound_merge_numerical['truck_signin_datetime_inbound'].apply(lambda x: time.mktime(x.timetuple()))
outbound_merge_numerical['truck_signin_datetime_outbound'] = outbound_merge_numerical['truck_signin_datetime_outbound'].apply(lambda x: time.mktime(x.timetuple()))

weather_post.sort_values("datetime_america", inplace=True)
weather_merge = pd.merge_asof(outbound_merge, weather_post, left_on='datetime_local', right_on='datetime_america', direction='nearest')

weather_merge_numerical = weather_merge.copy()
weather_merge_numerical['datetime_local'] = weather_merge_numerical['datetime_local'].apply(lambda x: time.mktime(x.timetuple()))
weather_merge_numerical['truck_signin_datetime_inbound'] = weather_merge_numerical['truck_signin_datetime_inbound'].apply(lambda x: time.mktime(x.timetuple()))
weather_merge_numerical['truck_signin_datetime_outbound'] = weather_merge_numerical['truck_signin_datetime_outbound'].apply(lambda x: time.mktime(x.timetuple()))
weather_merge_numerical['datetime_america'] = weather_merge_numerical['datetime_america'].apply(lambda x: time.mktime(x.timetuple()))

no_dates_merged = weather_merge_numerical.drop(['datetime_local', 'truck_signin_datetime_inbound', 'truck_signin_datetime_outbound', 'datetime_america'], axis=1)
nonan_val = no_dates_merged.dropna().reset_index(drop=True)

nonan_val.to_csv('preprocessed_data/preprocessed_val_data.csv', index=False)

In [18]:
print(f'Train set: {nonan_train.shape}')
print(f'Validation set: {nonan_val.shape}')

Train set: (190966, 19)
Validation set: (82192, 19)
